# Load the model

In [1]:
MODEL_PATH = 'pyannote/segmentation-3.0'
MY_TOKEN = None


PYDB_PATH="/home/aplaquet/work58/databases/msdwild/database.yml"
PROTOCOL="MSDWILD.SpeakerDiarization.OriginalMany"
PROTOCOL_SUBSET='test'

In [2]:
from pyannote.audio import Model


model = Model.from_pretrained(MODEL_PATH, use_auth_token=MY_TOKEN)

In [3]:
from pyannote.database import registry, FileFinder

registry.load_database(PYDB_PATH)
protocol = registry.get_protocol(PROTOCOL, preprocessors={"audio": FileFinder()})

# Generate inference file

In [4]:
from powerset_calibration.inference import get_inference_fid, model_inference_to_file

output_identifier = get_inference_fid('segmentation', PROTOCOL, PROTOCOL_SUBSET)
model_inference_to_file(
    model=model,
    protocol=protocol.test(),
    output_identifier=output_identifier,
    outputs_folder='outputs',
    step_ratio=0.5,
)

Inference started at 2024-06-10 11:05:48


100%|██████████| 177/177 [00:11<00:00, 16.05it/s, (11:06:00) 03141]


Inference done at 2024-06-10 11:06:00. Saving ...


# Evaluate inference file


In [10]:
from powerset_calibration.evaluation import DerComponentsDict, compute_der_inference_file
from pathlib import Path

stats: DerComponentsDict = compute_der_inference_file(
    fid=Path('outputs') / output_identifier
)

print(f'Local DER: {stats["der"]*100:.2f} % (FA: {stats["false_alarm"]*100:.2f} %, MISS: {stats["missed_detection"]*100:.2f} %, CONF: {stats["speaker_confusion"]*100:.2f} %)')

Local DER: 36.62 % (FA: 9.40 %, MISS: 13.89 %, CONF: 13.33 %)
